# **Judging the Influence of the Diamondback's Reporting- An Analysis of Prince George's County's 2018 - 2020 Food Inspection Data**

In [67]:
#importing necessary libraries
from google.colab import files
import pandas as pd
import io
import numpy as np
import plotly.express as px

In [3]:
uploaded = files.upload()   #uploading file to colab

Saving food_inspection_2018_2020 - food_inspection_2018_2020.csv to food_inspection_2018_2020 - food_inspection_2018_2020.csv


In [4]:
#representing file as a pandas dataframe
data_file = 'food_inspection_2018_2020 - food_inspection_2018_2020.csv'

df = pd.read_csv(io.BytesIO(uploaded[data_file]))
df  #viewing data

,Establishment_id,Name,Category,Inspection_date,Inspection_results,City,State,Zip,Address_line_1,Address_line_2,...,Hot_and_cold_running_water_provided,Proper_sewage_disposal,No_bare_hand_contact,Adequate_hand_washing_facilities,Rodent_and_insects,Food_contact_surfaces_and_equipment,Inspection_type,Owner,Type,Location
0,1146,PIZZA HUT 23538,Fast Food - Chain,5/29/2020 0:00,Non-Compliant - Violations Observed,COLLEGE PARK,MD,20740,9015 BALTIMORE AVE,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Change Of Ownership,LUIS IBARGUENGOYTIA,Food Establishments,POINT (-76.931327 38.999878)
1,2196,SEVEN ELEVEN 11664C,Convenience Store,7/9/2020 0:00,Compliance Schedule - Outstanding,COLLEGE PARK,MD,20740,4404 KNOX RD,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,RUCHEE JUDGE,Food Establishments,POINT (-76.938659 38.980959)
2,342,COLLEGE PARK DINER,Restaurant,7/17/2020 0:00,------,COLLEGE PARK,MD,20740,9206 BALTIMORE AVE,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,NaN,Adel RIAD,Food Establishments,POINT (-76.929936 39.006215)
3,2228,SEVEN ELEVEN 11642B,Convenience Store,3/16/2020 0:00,Non-Compliant - Violations Observed,COLLEGE PARK,MD,20740,4921 EDGEWOOD RD,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Comprehensive,EABA YADOTA,Food Establishments,POINT (-76.922405 39.015982)
4,730,JASON'S DELI,Fast Food - Chain,3/13/2020 0:00,Compliance Schedule - Outstanding,COLLEGE PARK,MD,20740,7356 BALTIMORE AVE,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,JOE TORTORICE JR,Food Establishments,POINT (-76.937981 38.980544)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630,7504,SUBWAY SANDWICH AND SALAD RESTAURANT,Fast Food,9/4/2019 0:00,Critical Violations observed,COLLEGE PARK,MD,20740,3711 CAMPUS DR,E,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Comprehensive,MOHIUDDIN KADER,Food Establishments,POINT (-76.949424 38.984745)
2631,1604,UMCP - MULLIGANS GRILL,College/University,9/4/2019 0:00,Compliance Schedule - Outstanding,COLLEGE PARK,MD,20742,3841 CAMPUS DR,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Monitoring,JOSEPH MULLINEAUX,Food Establishments,POINT (-76.948226 38.984828)
2632,272,CHECKERS DRIVE THRU RESTAURANT,Carry-out,9/3/2019 0:00,Non-Compliant - Violations Observed,COLLEGE PARK,MD,20740,6051 GREENBELT RD,------,...,In Compliance,In Compliance,In Compliance,In Compliance,Out of Compliance,In Compliance,Comprehensive,DANA KEITH,Food Establishments,POINT (-76.910024 38.997563)
2633,2339,UMCP - MD STADIUM UPPER CONCOURSE STANDS 302-312,College/University,8/12/2019 0:00,Non-Compliant - Violations Observed,COLLEGE PARK,MD,20742,3841 CAMPUS DR,------,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Pre-Opening Inspection,JOSEPH MULLINEAUX,Food Establishments,POINT (-76.948226 38.984828)


As the task asks to find a story from the 2018-2020 data, we must filter it accordingly.

In [97]:
df['Inspection_date'] = pd.to_datetime(df['Inspection_date'])   #converting Inspection_date column to pandas datetime 
mask = (df['Inspection_date'] >= '2018-01-01') & (df['Inspection_date'] < '2021-01-01') #creating a data mask to filter for only inspections conducted between 2018 and 2020

res_df = df[mask] #masked data 
res_df = res_df.drop(['City', 'State', 'Zip', 'Address_line_1', 'Address_line_2', 'Type'], axis=1) #dropping columns which have common values across all labels

res_df.sort_values(by=['Inspection_date']).shape[0] #checking to see how many rows remain

789

In [98]:
dropped_indices = res_df.index[res_df['Location'].isnull()].tolist()    #Finding locations with no coordinates available
res_df = res_df.drop(dropped_indices)   #removing said rows

res_df['Longitudes'] = [float(i.split()[1][1:]) for i in res_df['Location'].values] #splitting longitudes from each coordinate available and adding it to a new column called Longitudes
res_df['Latitudes'] = [float(i.split()[2][:-1]) for i in res_df['Location'].values] #splitting latitudes from each coordinate available and adding it to a new column called Latitudes

res_df = res_df.drop('Location', axis=1)    #removing location column (not of use anymore)
res_df.head()   #checking to see if changes are reflected (top 5 rows enough to check)

,Establishment_id,Name,Category,Inspection_date,Inspection_results,Food_from_approved_source,Food_protected_from_contamination,Ill_workers_restricted,Proper_hand_washing,Cooling_time_and_temperature,...,Hot_and_cold_running_water_provided,Proper_sewage_disposal,No_bare_hand_contact,Adequate_hand_washing_facilities,Rodent_and_insects,Food_contact_surfaces_and_equipment,Inspection_type,Owner,Longitudes,Latitudes
0,1146,PIZZA HUT 23538,Fast Food - Chain,2020-05-29,Non-Compliant - Violations Observed,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Change Of Ownership,LUIS IBARGUENGOYTIA,-76.931327,38.999878
1,2196,SEVEN ELEVEN 11664C,Convenience Store,2020-07-09,Compliance Schedule - Outstanding,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,RUCHEE JUDGE,-76.938659,38.980959
2,342,COLLEGE PARK DINER,Restaurant,2020-07-17,------,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,NaN,Adel RIAD,-76.929936,39.006215
3,2228,SEVEN ELEVEN 11642B,Convenience Store,2020-03-16,Non-Compliant - Violations Observed,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Comprehensive,EABA YADOTA,-76.922405,39.015982
4,730,JASON'S DELI,Fast Food - Chain,2020-03-13,Compliance Schedule - Outstanding,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,JOE TORTORICE JR,-76.937981,38.980544


In [99]:
#Ranking types of violations, 5 being worst, 1 being best, 0 being irrelevant
insp_res_dict = {'------':0,
                 'Facility Closed':0,
                 'Facility Reopened':0,
                 'Critical Violations observed':5,
                 'Non-Compliant - Violations Observed':4,
                 'Compliant - Health Risk':3,
                 'Compliant - No Health Risk':2,
                 'Compliance Schedule - Completed':2,
                 'Compliance Schedule - Outstanding':1}

#creating list with values of every locations's inspection result according to the aforementioned rank system
insp_res_scale = []
for i in (res_df['Inspection_results'].values):
    insp_res_scale.append(insp_res_dict[i])

res_df['Inspection_result_scale'] = insp_res_scale  #assigning list to a new column called Inspection_result_scale
res_df.head()   #checking to see if changes are reflected (top 5 rows enough to check)

,Establishment_id,Name,Category,Inspection_date,Inspection_results,Food_from_approved_source,Food_protected_from_contamination,Ill_workers_restricted,Proper_hand_washing,Cooling_time_and_temperature,...,Proper_sewage_disposal,No_bare_hand_contact,Adequate_hand_washing_facilities,Rodent_and_insects,Food_contact_surfaces_and_equipment,Inspection_type,Owner,Longitudes,Latitudes,Inspection_result_scale
0,1146,PIZZA HUT 23538,Fast Food - Chain,2020-05-29,Non-Compliant - Violations Observed,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Change Of Ownership,LUIS IBARGUENGOYTIA,-76.931327,38.999878,4
1,2196,SEVEN ELEVEN 11664C,Convenience Store,2020-07-09,Compliance Schedule - Outstanding,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,RUCHEE JUDGE,-76.938659,38.980959,1
2,342,COLLEGE PARK DINER,Restaurant,2020-07-17,------,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,NaN,Adel RIAD,-76.929936,39.006215,0
3,2228,SEVEN ELEVEN 11642B,Convenience Store,2020-03-16,Non-Compliant - Violations Observed,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Comprehensive,EABA YADOTA,-76.922405,39.015982,4
4,730,JASON'S DELI,Fast Food - Chain,2020-03-13,Compliance Schedule - Outstanding,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,...,In Compliance,In Compliance,In Compliance,In Compliance,In Compliance,Re-inspection,JOE TORTORICE JR,-76.937981,38.980544,1


Now we can use the data we have parsed to visualize it on a live map using the polt.ly library

In [142]:
#using the plot.ly library to visualize each location on a map to see any correlations
def disp_map(df1):  #function takes in a dataframe (table) and outputs a map with coordiantes according to lats and longs
    fig = px.scatter_mapbox(df1, lat="Latitudes", lon="Longitudes", size='Inspection_result_scale', color_discrete_sequence=["fuchsia"],
                color_continuous_scale=px.colors.cyclical.IceFire, size_max=5, zoom=12)

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [263]:
disp_map(res_df)    #visualizing all of the locations of inspection

In [264]:
worst_offenders = res_df[res_df['Inspection_result_scale'] == 5]    #viewing all of the worst offenders on a map
disp_map(worst_offenders)

The above maps are not very conclusive besides the fact that most of the worst offenders lie on Route 1- a claim that isn't conclusive as most of the eateries available in college park also are found in that general area.

Instead, we can look at the affect of reporting on this information and what it has done to improve the situation (if anything). I used the diamondback's article "Nando’s and Potbelly are among the College Park eateries cited for health code violations" written by Jeff Barnes in November of 2018 to measure this phenomenon.

In [262]:
offenders_control = set(res_df[(res_df['Inspection_result_scale'] >= 4) & (res_df['Inspection_date'] < '2018-11-06')]['Name'])  #set of offenders before Nov 6 2018
compliant_control = set(res_df[(res_df['Inspection_result_scale'] <= 2) & (res_df['Inspection_date'] < '2018-11-06')]['Name'])  #set of compliants before Nov 6 2018

offenders_before_story = set(res_df[(res_df['Inspection_result_scale'] >= 4) & (res_df['Inspection_date'] < '2018-11-06')]['Name']) - compliant_control
compliant_after_story = set(res_df[(res_df['Inspection_result_scale'] <= 2) & (res_df['Inspection_date'] > '2018-11-06') & (res_df['Inspection_date'] < '2019-10-06')]['Name'])

nonreformed_control = offenders_control.difference(compliant_control)   #Set of eateries not compliant before story
reformed_control = offenders_control.intersection(compliant_control)    #Set of eateries compliant before story

nonreformed_story = offenders_before_story.difference(compliant_after_story)    #Set of eateries not compliant before story WITHOUT eateries reformed before story publish date 
reformed_story = offenders_before_story.intersection(compliant_after_story)     #Set of eateries compliant after story upto next 11 months

print("Non Compliance to Compliance Metrics:")
print("Before story in 11 months:", len(offenders_control), "->", len(reformed_control), "|", "After story in 11 months:", len(offenders_before_story), "->", len(reformed_story))
# len(reformed_control)

Non Compliance to Compliance Metrics:
Before story in 11 months: 82 -> 28 | After story in 11 months: 54 -> 30


It is a lot more visible that effective reporting on eateries and their violations increases exposure to reputational risk and speeds up the compliance process. This is in no way a sure-fire way to prove this assumption however shows evidence of a potential underlying truth.

In [265]:
continued_offenders = res_df[res_df['Name'].apply(lambda x: x in nonreformed_story)]    #viewing all of the continued offenders after the story
disp_map(continued_offenders)

This map also demonstrates a lack of reach/care of the Diamondback's reporting, with a majority of the continued offenders found to beon Route 1. This could be indicative of an indifferent attitude of owners in the area. 